In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Topic Modelling**

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 84.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models.phrases import Phrases, Phraser

In [ ]:
# Load data from Part 1
df_preprocessed = pd.read_csv("/content/drive/MyDrive/IST 332 NLP Final Project/urgent_care_reviews_CA_fully_preprocessed.csv")
df_preprocessed.head()

,place_id,place_name,place_address,place_avg_rating,place_user_ratings_total,place_types,review_rating,review_text,review_relative_time_description,expanded_review,review_tokens,normalized_review,review_tagged,review_lemmas,review_lemmas_expanded,review_lemmas_expanded_cleaned,review_cleaned_text
0,ChIJbSt9IZDDlIARo2FxybKYSSc,Omni Family Health | Hanford Health Center,"355 Campus Dr suite e, Hanford, CA 93230, USA",4.0,69,"doctor,establishment,health,point_of_interest",5,I love this place. They treat you like a human...,2 months ago,I love this place. They treat you like a human...,"['I', 'love', 'this', 'place', '.', 'They', 't...","['love', 'place', 'treat', 'like', 'human', 'n...","[('love', 'VB'), ('place', 'NN'), ('treat', 'N...","['love', 'place', 'treat', 'like', 'human', 'n...","['love', 'place', 'treat', 'like', 'human', 'n...","['love', 'place', 'treat', 'like', 'human', 'n...",love place treat like human number really take...
1,ChIJbSt9IZDDlIARo2FxybKYSSc,Omni Family Health | Hanford Health Center,"355 Campus Dr suite e, Hanford, CA 93230, USA",4.0,69,"doctor,establishment,health,point_of_interest",1,I had an appointment on the 22nd for an ear in...,4 months ago,I had an appointment on the 22nd for an ear in...,"['I', 'had', 'an', 'appointment', 'on', 'the',...","['appointment', 'ear', 'infection', 'prescribe...","[('appointment', 'NN'), ('ear', 'JJ'), ('infec...","['appointment', 'ear', 'infection', 'prescribe...","['appointment', 'ear', 'infection', 'prescribe...","['appointment', 'ear', 'infection', 'prescribe...",appointment ear infection prescribe antibiotic...
2,ChIJbSt9IZDDlIARo2FxybKYSSc,Omni Family Health | Hanford Health Center,"355 Campus Dr suite e, Hanford, CA 93230, USA",4.0,69,"doctor,establishment,health,point_of_interest",5,Dr. Garza is just amazing Takes time to hear ...,7 months ago,Dr. Garza is just amazing Takes time to hear a...,"['Dr.', 'Garza', 'is', 'just', 'amazing', 'Tak...","['dr', 'garza', 'amazing', 'takes', 'time', 'h...","[('dr', 'NN'), ('garza', 'NN'), ('amazing', 'V...","['dr', 'garza', 'amaze', 'take', 'time', 'hear...","['doctor', 'garza', 'amaze', 'take', 'time', '...","['doctor', 'amaze', 'take', 'time', 'hear', 'w...",doctor amaze take time hear week make goal ski...
3,ChIJbSt9IZDDlIARo2FxybKYSSc,Omni Family Health | Hanford Health Center,"355 Campus Dr suite e, Hanford, CA 93230, USA",4.0,69,"doctor,establishment,health,point_of_interest",5,Staff is always very polite. I have never expe...,a year ago,Staff is always very polite. I have never expe...,"['Staff', 'is', 'always', 'very', 'polite', '....","['staff', 'always', 'polite', 'never', 'experi...","[('staff', 'NN'), ('always', 'RB'), ('polite',...","['staff', 'always', 'polite', 'never', 'experi...","['staff', 'always', 'polite', 'never', 'experi...","['always', 'polite', 'never', 'experience', 'w...",always polite never experience wait time call ...
4,ChIJbSt9IZDDlIARo2FxybKYSSc,Omni Family Health | Hanford Health Center,"355 Campus Dr suite e, Hanford, CA 93230, USA",4.0,69,"doctor,establishment,health,point_of_interest",5,Dr. Thomas Warner is the BEST doctor I have ev...,2 years ago,Dr. Thomas Warner is the BEST doctor I have ev...,"['Dr.', 'Thomas', 'Warner', 'is', 'the', 'BEST...","['dr', 'thomas', 'warner', 'best', 'doctor', '...","[('dr', 'NN'), ('thomas', 'NN'), ('warner', 'N...","['dr', 'thomas', 'warner', 'best', 'doctor', '...","['doctor', 'thomas', 'warner', 'best', 'doctor...","['doctor', 'warner', 'best', 'doctor', 'see', ...",doctor warner best doctor see make health conc...


In [ ]:
# Prepare Text for LDA
# Function to convert message_lemmas to list of tokens
def process_message_lemmas(message_lemmas):
    """Convert message lemmas to list of tokens"""
    if isinstance(message_lemmas, str):
        if message_lemmas.startswith('['):
            import ast
            try:
                return ast.literal_eval(message_lemmas)
            except:
                return message_lemmas.split()
        else:
            return message_lemmas.split()
    elif isinstance(message_lemmas, list):
        return message_lemmas
    else:
        return str(message_lemmas).split()

In [ ]:
# Create list of tokenized documents
texts = [process_message_lemmas(doc) for doc in df_preprocessed['review_cleaned_text']]

In [ ]:
print(texts[0][:10])

['love', 'place', 'treat', 'like', 'human', 'number', 'really', 'take', 'good', 'care']


In [ ]:
# Create Bigrams
# Train bigram model
# min_count: minimum frequency threshold
min_count_threshold = max(5, int(len(texts) * 0.01))  # 1% of documents or 5
print(f"Using min_count threshold: {min_count_threshold}")

bigram = Phrases(texts, min_count=min_count_threshold)
bigram_phraser = Phraser(bigram)

Using min_count threshold: 116


In [ ]:
# Apply bigram model to texts
texts_bigram = [bigram_phraser[doc] for doc in texts]

In [ ]:
# Count bigrams created
bigram_count = sum(1 for doc in texts_bigram for token in doc if '_' in token)


In [ ]:
# Create Dictionary and Corpus
# Build dictionary
id2word = Dictionary(texts_bigram)

In [ ]:
no_below = max(5, int(len(texts_bigram) * 0.01))  # 1% or 5 documents
no_above = 0.75  # 75% of documents

In [ ]:
# Create corpus (Bag of Words)
corpus = [id2word.doc2bow(doc) for doc in texts_bigram]

In [ ]:
print(corpus[0][:5])

[(0, 1), (1, 2), (2, 2), (3, 1), (4, 1)]


In [ ]:
# Train Multiple LDA Models
topic_numbers = [5, 8, 10]

In [ ]:
import time

In [ ]:
# LDA hyperparameters
chunksize = 100
passes = 20
iterations = 400
eval_every = None

In [ ]:
# Store models
lda_models = {}


In [ ]:
for n_topics in topic_numbers:

    start_time = time.time()

    # Train LDA model
    lda_model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=n_topics,
        random_state=42,
        chunksize=chunksize,
        passes=passes,
        alpha='symmetric',
        eta='symmetric',
        iterations=iterations,
        gamma_threshold=0.001,
        per_word_topics=True,
        eval_every=eval_every
    )

    training_time = time.time() - start_time

    # Store the model
    lda_models[n_topics] = lda_model

In [ ]:
print(f"Training completed in {training_time:.2f} seconds")

Training completed in 82.67 seconds


In [ ]:
lda_models

{3: <gensim.models.ldamodel.LdaModel at 0x7dab479be9c0>,
 5: <gensim.models.ldamodel.LdaModel at 0x7dab4b6e2ab0>,
 7: <gensim.models.ldamodel.LdaModel at 0x7dab479be480>}

In [ ]:
# Save the processed data for evaluation
import pickle

# Save corpus and dictionary
with open('lda_corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)

with open('lda_id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)

with open('lda_texts.pkl', 'wb') as f:
    pickle.dump(texts_bigram, f)

In [ ]:
# Save all models to be used for part 6.2
print(f"Saving {len(lda_models)} models...")
for n_topics, model in lda_models.items():
    filepath = f'/content/drive/MyDrive/Colab Notebooks/lda_model_{n_topics}topics.model'
    model.save(filepath)
    print(f"Saved model with {n_topics} topics")


Saving 3 models...
Saved model with 3 topics
Saved model with 5 topics
Saved model with 7 topics


In [ ]:
import pandas as pd
import numpy as np
import pickle
import gensim
from gensim.models import LdaModel, CoherenceModel
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Saved Models and Data
# Load corpus and dictionary
with open('lda_corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)

with open('lda_id2word.pkl', 'rb') as f:
    id2word = pickle.load(f)

with open('lda_texts.pkl', 'rb') as f:
    texts = pickle.load(f)

print("Data summary:")
print("Corpus size: ", len(corpus))
print("Vocabulary size: ", len(id2word))



Data summary:
Corpus size:  11658
Vocabulary size:  15816


In [ ]:
# Load models
topic_numbers = [3, 5, 7]
lda_models = {}

In [ ]:
for n_topics in topic_numbers:
        lda_models[n_topics] = LdaModel.load(f'/content/drive/MyDrive/Colab Notebooks/lda_model_{n_topics}topics.model')
        print(f"Loaded model with {n_topics} topics")

Loaded model with 3 topics
Loaded model with 5 topics
Loaded model with 7 topics


In [ ]:
# Top 10 Words for Each Topic
all_topics_info = {}

for n_topics in topic_numbers:
    model = lda_models[n_topics]

    # Print model header with clear separation

    print(f"MODEL: LDA with {n_topics} Topics")


    # Get topics (words)
    topics_list = []

    for topic_idx in range(n_topics):
        # Get topic terms with probabilities (top 10)
        topic_terms = model.show_topic(topic_idx, topn=10)
        words = [word for word, _ in topic_terms]
        weights = [weight for _, weight in topic_terms]

        topics_list.append({
            'words': words

        })

        # Display topic with clean formatting
        print(f"Topic {topic_idx + 1}:")
        print(f"  Top 10 words: {', '.join(words)}")


        print()
    print("======================")
    all_topics_info[n_topics] = topics_list




MODEL: LDA with 3 Topics
Topic 1:
  Top 10 words: care, doctor, experience, time, great, recommend, medical, take, professional, patient

Topic 2:
  Top 10 words: doctor, go, tell, get, say, call, would, patient, ask, give

Topic 3:
  Top 10 words: wait, time, go, get, appointment, see, urgent_care, doctor, take, hour

MODEL: LDA with 5 Topics
Topic 1:
  Top 10 words: area, clearly, waste_time, explain_everything, far, dental, client, life, option, future

Topic 2:
  Top 10 words: go, get, call, tell, say, doctor, would, appointment, ask, see

Topic 3:
  Top 10 words: time, doctor, urgent_care, wait, see, great, go, need, experience, take

Topic 4:
  Top 10 words: pain, emergency_room, pay, hospital, nothing, give, referral, life, less, unprofessional

Topic 5:
  Top 10 words: care, doctor, patient, take, medical, health, experience, well, concern, make

MODEL: LDA with 7 Topics
Topic 1:
  Top 10 words: comfortable, process, area, clearly, easy, fast, explain_everything, dental, go_bey

In [ ]:
# Topic Coherence Metrics

coherence_scores = {}

for n_topics in topic_numbers:
    model = lda_models[n_topics]

    # Calculate coherence
    coherence_model = CoherenceModel(
        model=model,
        texts=texts,
        dictionary=id2word,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    coherence_scores[n_topics] = coherence_score

    print(f"{n_topics}-topic model:")

    print(f"  Coherence (C_v): {coherence_score:.4f}")



3-topic model:
  Coherence (C_v): 0.4027
5-topic model:
  Coherence (C_v): 0.3423
7-topic model:
  Coherence (C_v): 0.3517


In [ ]:
# Save evaluation results
results = {
    'coherence_scores': coherence_scores,
    'all_topics': all_topics_info
}

In [ ]:
with open('evaluation_results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
with open('evaluation_results.pkl', 'rb') as f:
    results = pickle.load(f)

coherence_scores = results['coherence_scores']
all_topics = results['all_topics']

In [ ]:
best_num_topics = max(coherence_scores, key=coherence_scores.get)
best_coherence = coherence_scores[best_num_topics]
best_model = lda_models[best_num_topics]
best_topics_info = all_topics[best_num_topics]


In [ ]:
# Display all coherence scores
print(f"Coherence Scores Comparison:")
for n_topics in sorted(coherence_scores.keys()):
    print(f"  {n_topics} topics: {coherence_scores[n_topics]:.4f}")
#select the best model based on the coherence score (higher score is better)
print(f" Best model has {best_num_topics} topics with coherence score: {best_coherence:.4f}")

Coherence Scores Comparison:
  3 topics: 0.4027
  5 topics: 0.3423
  7 topics: 0.3517
 Best model has 3 topics with coherence score: 0.4027


In [ ]:
# Display the top 10 for the best model

for topic_idx in range(best_num_topics):
    # Get topic information
    topic_info = best_topics_info[topic_idx]
    top_words = topic_info['words'][:10]  # Top 10 words
    words_str = ', '.join(top_words)


    print(f"Topic {topic_idx + 1} ")

    print(f"Top 10 Words: {words_str}")


Topic 1 
Top 10 Words: care, doctor, experience, time, great, recommend, medical, take, professional, patient
Topic 2 
Top 10 Words: doctor, go, tell, get, say, call, would, patient, ask, give
Topic 3 
Top 10 Words: wait, time, go, get, appointment, see, urgent_care, doctor, take, hour
